In [1]:
import sys
sys.path.append('..')

In [11]:
from llm_wrapper import get_chat_llm, get_embeddings
from pdf_parsing import VectorStoreManager, RAGAgent
from langchain.agents import create_agent
from langchain.tools import tool
from db_agent import SQLAgent

In [33]:
from opik.integrations.langchain import OpikTracer 

In [3]:
embeddings = get_embeddings()
llm = get_chat_llm()

✓ Opik tracer initialized for project: customer-support


In [5]:
vector_store_manager = VectorStoreManager(
    embeddings=embeddings,
    collection_name="smart_scribble_docs"
)

rag_agent = RAGAgent(llm, vector_store_manager, k=3)

In [9]:
rag_agent.agent.invoke({
    'messages': ['give me technical spefications of the product in 30 words']
})

{'messages': [HumanMessage(content='give me technical spefications of the product in 30 words', additional_kwargs={}, response_metadata={}, id='87d5ab01-1d14-4a59-bbf7-43f7c5116c6a'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 99, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CjeMJdheJfY7gCfbNlkdjqMG8P2v7', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence':

In [15]:
sql_agent = SQLAgent(llm, dialect="Postgres", top_k=5)
sql_agent.agent.invoke({
    'messages': ['summarize the feedbacks of product with ratings less than or equal to 3.']
})

{'messages': [HumanMessage(content='summarize the feedbacks of product with ratings less than or equal to 3.', additional_kwargs={}, response_metadata={}, id='cc350566-a971-43e0-9566-0f2e816edf6c'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 562, 'total_tokens': 575, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CjeTSWnn7biqTcnOVMEoVHCGXQNia', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'saf

In [36]:
from langgraph.types import Command, interrupt

@tool
def rag_tool(query: str) -> str:
    """Tool to answer product related queries using RAG agent."""
    return rag_agent.agent.invoke({'messages': [query]})['messages'][-1].content

@tool
def sql_tool(query: str) -> str:
    """Tool to answer transactions questions from the user, this tool will be able to generate the sql query and respond
        query -> str : user query ex. 'summarize the feedbacks of product with ratings less than or equal to 3.'
    """
    return sql_agent.agent.invoke({'messages': [query]})['messages'][-1].content


@tool
def get_human_feedback(query: str) -> str:
    """Tool to get human feedback for queries not answered by other agents or tools."""
    response = interrupt({
        "type": "human_feedback",
        "query": query
    })
    return response

In [39]:
from langgraph.checkpoint.memory import InMemorySaver

supervisor_agent = create_agent(
    llm, 
    tools=[rag_tool, sql_tool, get_human_feedback],
    checkpointer=InMemorySaver(),
    system_prompt="You are a helpful customer support assistant. Use the available tools to answer user queries effectively. If unsure or not getting relevant response from existing tools, seek human feedback."
    )

In [34]:
tracer = OpikTracer(graph=supervisor_agent.get_graph(xray=True)) 

In [35]:
repsonse = supervisor_agent.invoke({
    'messages': ['Provide an overview of the technical specifications and summarize customer feedback from the database.']
}, config={
        "callbacks": [tracer], 
    })

OPIK: You are attempting to log data into a nested span under the project name "customer-support". However, the project name "Default Project" from parent span will be used instead.
OPIK: You are attempting to log data into a nested span under the project name "customer-support". However, the project name "Default Project" from parent span will be used instead.
OPIK: You are attempting to log data into a nested span under the project name "customer-support". However, the project name "Default Project" from parent span will be used instead.
OPIK: You are attempting to log data into a nested span under the project name "customer-support". However, the project name "Default Project" from parent span will be used instead.
OPIK: You are attempting to log data into a nested span under the project name "customer-support". However, the project name "Default Project" from parent span will be used instead.
OPIK: You are attempting to log data into a nested span under the project name "customer-s

In [32]:
print(repsonse['messages'][-1].content)

The device has the following technical specifications:
- 10.3-inch CanvasInk™ E-Ink Display (Monochrome) with a resolution of 1872 × 1404 (227 DPI)
- Quad-Core 1.8GHz ARM Cortex Processor
- 64GB Internal Storage (Non-expandable)
- 4GB LPDDR4X RAM
- Connectivity options include Wi-Fi (2.4GHz + 5GHz), Bluetooth 5.0, and USB-C
- 3000mAh Li-ion Battery offering up to 2 weeks standby and 10 hours of active writing
- Wacom EMR Technology Stylus (Battery-free) with 4096 Pressure Levels
- Dimensions: 230mm × 190mm × 5.8mm; Weight: 395g
- OS: SmartOS (Based on Linux Kernel)
- Supports file formats such as PDF, EPUB, TXT, PNG, and SVG
- Additional features include aerospace-grade aluminum alloy chassis, bi-directional sync with platforms like Google Drive, OneNote, Evernote, Dropbox, and OneDrive

Customer feedback summary:
- Mostly high ratings (4-5 stars) praising features
- Positive feedback on pressure sensitivity for artists, useful AI features, effective OCR, and overall performance
- Appr

In [40]:
config = {"configurable": {"thread_id": "1"}, "callbacks": [tracer]}

response = supervisor_agent.invoke({
    'messages': ['what is the weather in mumbai']
}, config=config)

OPIK: You are attempting to log data into a nested span under the project name "customer-support". However, the project name "Default Project" from parent span will be used instead.


In [41]:
response

{'messages': [HumanMessage(content='what is the weather in mumbai', additional_kwargs={}, response_metadata={}, id='26864288-d5bc-44d4-8fd6-f810be479535'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 183, 'total_tokens': 206, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CjflxVVfvG7jwW2PPM7mdi4AxdQQY', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'sever

In [42]:
response = supervisor_agent.invoke(
    Command(resume="We do not support this query")
    , config=config)
response

OPIK: You are attempting to log data into a nested span under the project name "customer-support". However, the project name "Default Project" from parent span will be used instead.


{'messages': [HumanMessage(content='what is the weather in mumbai', additional_kwargs={}, response_metadata={}, id='26864288-d5bc-44d4-8fd6-f810be479535'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 183, 'total_tokens': 206, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CjflxVVfvG7jwW2PPM7mdi4AxdQQY', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'sever

In [ ]:
user ui -> query
query -> backend server
backend server -> agent
backend server -> human agent server
human agent server -> backend server
backend server -> agent (human agent feedback)
backend server -> user ui (final response)